In [33]:
# Importing packages
import pandas as pd
import discogs_client
import re
import time
import requests
import json
import os
from dotenv import load_dotenv

# Discogs

In [34]:
# Client
load_dotenv()

discogs = discogs_client.Client('Discogs_to_Notion', user_token=os.getenv('discogs_token'))
me = discogs.identity()

## Release ids of Discogs wantlist and collection

In [35]:
# Discogs wantlist
wantlist = me.wantlist
wantlist_ids = []
for item in wantlist:
    print(item)
    wantlist_ids.append(item.id)

<WantlistItem 13835553 'Ventura'>
<WantlistItem 13518819 'Not Waving, But Drowning'>
<WantlistItem 20846650 'Day/Night'>
<WantlistItem 4570366 'Random Access Memories'>
<WantlistItem 12448883 'Kingdoms In Colour'>
<WantlistItem 24470483 'No Sleep in LA'>
<WantlistItem 15511221 'Mordechai'>
<WantlistItem 24861332 'Hugo'>
<WantlistItem 7252111 'Currents'>
<WantlistItem 21709576 'Fragments'>
<WantlistItem 10435568 'Take a Rest'>
<WantlistItem 8827787 'Malibu'>
<WantlistItem 15461907 'Bubba'>
<WantlistItem 7557957 'To Pimp A Butterfly'>
<WantlistItem 21483991 'Can‘t Buy The Mood'>
<WantlistItem 7033756 'Walk Dance Talk Sing'>
<WantlistItem 15389727 'Circles'>
<WantlistItem 5877431 'Jungle'>


In [36]:
# Discogs collection
collection = me.collection_folders[0].releases
collection_ids = []
for item in collection:
    print(item)
    # Add release ids to list
    collection_ids.append(item.id)

<CollectionItemInstance 13113693 'Oxnard'>
<CollectionItemInstance 20095591 'Sometimes I Might Be Introvert'>
<CollectionItemInstance 12646784 'Parcels'>
<CollectionItemInstance 15222265 'Live Vol.1'>
<CollectionItemInstance 1183731 'Alligator'>
<CollectionItemInstance 14791710 'Suddenly'>
<CollectionItemInstance 19292665 'Cyclorama'>
<CollectionItemInstance 17526067 'Terra Firma'>
<CollectionItemInstance 13374359 'Significant Changes'>
<CollectionItemInstance 17030532 'Isles'>
<CollectionItemInstance 25286800 'Capricorn Sun'>
<CollectionItemInstance 24356900 'Mr. Morale & The Big Steppers'>
<CollectionItemInstance 17593180 'Sand'>
<CollectionItemInstance 4618675 'Settle'>
<CollectionItemInstance 12009384 'Wide Awake!'>
<CollectionItemInstance 14380475 'Are We Still Dreaming?'>
<CollectionItemInstance 7251550 'Venice'>
<CollectionItemInstance 26294105 'Remember'>
<CollectionItemInstance 2879 'Discovery'>
<CollectionItemInstance 14775292 'Cut The Wire '>
<CollectionItemInstance 12509101

## Get album info from Discogs

In [37]:
def get_album_info(release_ids, tag):
    # Initialize empty lists
    albums = []
    artists = []
    urls = []
    images = []

    for i in release_ids:
        release = discogs.release(i)                                                    # initiate release object using id
        album_title = release.title                                                     # album title
        artist_name = release.artists                                                   # artist name
        artist_filter = re.sub(r"^\[<Artist \d+ '(.*)'>\]$", r"\1", str(artist_name))   # filter artist name from list
        artist_filter = re.sub(r"\d+", "", artist_filter)
        artist_filter = artist_filter.replace("()", "")
        album_url = release.url                                                         # url
        album_image = release.images                                                    # image
        album_image = album_image[0]
        album_image = album_image.get('uri')                                                                                      
        
        # Append info to lists
        albums.append(album_title)
        artists.append(artist_filter)
        urls.append(album_url)
        images.append(album_image)

        # Print to see progress
        print(release)
        time.sleep(0.5)

    # Store info in df
    df = pd.DataFrame(list(zip(albums, artists, urls, images)),
                                    columns = ['album', 'artist', 'url', 'image'])
    
    if tag == 'wish':
        df['tags'] = 'wish'
    elif tag == 'collection':
        df['tags'] = 'collection'
    else:
        df['tags'] = None
                                    
    return df

In [38]:
discogs_collection = get_album_info(collection_ids, 'collection')
discogs_collection.head()

<Release 13113693 'Oxnard'>
<Release 20095591 'Sometimes I Might Be Introvert'>
<Release 12646784 'Parcels'>
<Release 15222265 'Live Vol.1'>
<Release 1183731 'Alligator'>
<Release 14791710 'Suddenly'>
<Release 19292665 'Cyclorama'>
<Release 17526067 'Terra Firma'>
<Release 13374359 'Significant Changes'>
<Release 17030532 'Isles'>
<Release 25286800 'Capricorn Sun'>
<Release 24356900 'Mr. Morale & The Big Steppers'>
<Release 17593180 'Sand'>
<Release 4618675 'Settle'>
<Release 12009384 'Wide Awake!'>
<Release 14380475 'Are We Still Dreaming?'>
<Release 7251550 'Venice'>
<Release 26294105 'Remember'>
<Release 2879 'Discovery'>
<Release 14775292 'Cut The Wire '>
<Release 12509101 'For Ever'>
<Release 8864483 '99.9%'>


,album,artist,url,image,tags
0,Oxnard,Anderson .Paak,https://www.discogs.com/release/13113693-Ander...,https://i.discogs.com/R9LcC7Y9Ij2vABwBhEGUtUvo...,collection
1,Sometimes I Might Be Introvert,Little Simz,https://www.discogs.com/release/20095591-Littl...,https://i.discogs.com/0SRVBvcx_63Td-IR9dHpVcEp...,collection
2,Parcels,Parcels,https://www.discogs.com/release/12646784-Parce...,https://i.discogs.com/9zhDlpy6cjuZw5RQV9ajAZwh...,collection
3,Live Vol.1,Parcels,https://www.discogs.com/release/15222265-Parce...,https://i.discogs.com/UQqD7a6KJUjy3flSn2xUlXfe...,collection
4,Alligator,The National,https://www.discogs.com/release/1183731-The-Na...,https://i.discogs.com/bWcIWn90Wb2iByGmmVIw-GrS...,collection


In [39]:
discogs_wantlist = get_album_info(wantlist_ids, 'wish')
discogs_wantlist.head()

<Release 13835553 'Ventura'>
<Release 13518819 'Not Waving, But Drowning'>
<Release 20846650 'Day/Night'>
<Release 4570366 'Random Access Memories'>
<Release 12448883 'Kingdoms In Colour'>
<Release 24470483 'No Sleep in LA'>
<Release 15511221 'Mordechai'>
<Release 24861332 'Hugo'>
<Release 7252111 'Currents'>
<Release 21709576 'Fragments'>
<Release 10435568 'Take a Rest'>
<Release 8827787 'Malibu'>
<Release 15461907 'Bubba'>
<Release 7557957 'To Pimp A Butterfly'>
<Release 21483991 'Can‘t Buy The Mood'>
<Release 7033756 'Walk Dance Talk Sing'>
<Release 15389727 'Circles'>
<Release 5877431 'Jungle'>


,album,artist,url,image,tags
0,Ventura,Anderson .Paak,https://www.discogs.com/release/13835553-Ander...,https://i.discogs.com/nRnMfcb_WpE80LhuwiAwPZQp...,wish
1,"Not Waving, But Drowning",Loyle Carner,https://www.discogs.com/release/13518819-Loyle...,https://i.discogs.com/2OgS_0_Iuy75TZ4x-FfeTuoO...,wish
2,Day/Night,Parcels,https://www.discogs.com/release/20846650-Parce...,https://i.discogs.com/c87RZSedRpAzG3GUpvgjj52F...,wish
3,Random Access Memories,Daft Punk,https://www.discogs.com/release/4570366-Daft-P...,https://i.discogs.com/zFVZE4s0zSXUIM7OMl2UDckS...,wish
4,Kingdoms In Colour,Maribou State,https://www.discogs.com/release/12448883-Marib...,https://i.discogs.com/izKlrBtyqrtHt80RP4CJvkbC...,wish


# Notion

In [40]:
# Initialization
token = os.getenv('notion_token')
database_id = '1afa86cc349c402ab660a19466400390'
headers = {
    'Authorization': 'Bearer ' + token,
    'Content-type': 'application/json',
    'Notion-Version': '2022-06-28'
}

## Get pages info (read)

In [41]:
def get_pages():
    read_url = f'https://api.notion.com/v1/databases/{database_id}/query'

    response = requests.post(read_url, headers=headers)

    data = response.json()
    with open('db.json', 'w', encoding='utf8') as f:    
        json.dump(data, f, ensure_ascii=False, indent=4)
    
    results = data['results']
    return results

In [42]:
pages = get_pages()

def get_pages_info(pages):

    albums = []
    artists = []
    urls = []
    tags = []
    page_ids = []
    images = []

    for page in pages:
       
        page_id = page['id']
        props = page['properties']
        album_title = props['Album']['title'][0]['text']['content']
        artist = props['Artist']['multi_select'][0]['name']
        album_url = props['URL']['url']
        tag = props['Tags']['multi_select'][0]['name']
        image = props['Album cover']['files'][0]['external']['url']

        # Append info to lists
        albums.append(album_title)
        artists.append(artist)
        urls.append(album_url)
        tags.append(tag)
        images.append(image)
        page_ids.append(page_id)

    df = pd.DataFrame(list(zip(albums, artists, urls, tags, images, page_ids)),
                                    columns = ['album', 'artist', 'url', 'tags', 'image', 'page_id'])
    
    return df

In [43]:
notion_pages = get_pages_info(pages)
notion_pages.head()

,album,artist,url,tags,image,page_id
0,Hugo,Loyle Carner,https://www.discogs.com/release/24861332-Loyle...,wish,https://i.discogs.com/sLP9Abw1ERF3n6A1sfsJt-J-...,7b0412e3-8777-4e3a-91b2-e0342a46c666
1,Ventura,Anderson .Paak,https://www.discogs.com/release/13835553-Ander...,wish,https://i.discogs.com/nRnMfcb_WpE80LhuwiAwPZQp...,76204dd1-0e46-4c30-b974-4dd2c63fc107
2,Take a Rest,Tora,https://www.discogs.com/release/10435568-Tora-...,wish,https://i.discogs.com/GMyKtIYZeuUDxQMdRQ3Q6lpg...,865201b5-af69-47c9-b9a3-9b567b7bc7df
3,Random Access Memories,Daft Punk,https://www.discogs.com/release/4570366-Daft-P...,wish,https://i.discogs.com/zFVZE4s0zSXUIM7OMl2UDckS...,d8aef602-f230-4f05-96c1-f3985eff397b
4,Bubba,Kaytranada,https://www.discogs.com/release/15461907-Kaytr...,wish,https://i.discogs.com/xyNZPWWPV5Hw3vnQHP-IMYyV...,03d33f30-bde0-4012-9f0f-61b907b6bb98


## Delete old pages

In [44]:
def delete_page(page_id: str):
    url = f"https://api.notion.com/v1/pages/{page_id}"

    payload = {"archived": True}

    res = requests.patch(url, json=payload, headers=headers)
    return res

In [45]:
for index, row in notion_pages.iterrows():
    page_id = row['page_id']
    album = row['album']

    print(f'Deleting {album} page...')
    delete_page(page_id)

Deleting Hugo page...
Deleting Ventura page...
Deleting Take a Rest page...
Deleting Random Access Memories page...
Deleting Bubba page...
Deleting Fragments page...
Deleting Walk Dance Talk Sing page...
Deleting To Pimp A Butterfly page...
Deleting Malibu page...
Deleting Jungle page...
Deleting Not Waving, But Drowning page...
Deleting Currents page...
Deleting Mordechai page...
Deleting No Sleep in LA page...
Deleting Can‘t Buy The Mood page...
Deleting Day/Night page...
Deleting Kingdoms In Colour page...
Deleting Circles page...
Deleting Wide Awake! page...
Deleting 99.9% page...
Deleting Live Vol.1 page...
Deleting Capricorn Sun page...
Deleting For Ever page...
Deleting Oxnard page...
Deleting Sand page...
Deleting Cut The Wire  page...
Deleting Are We Still Dreaming? page...
Deleting Mr. Morale & The Big Steppers page...
Deleting Venice page...
Deleting Remember page...
Deleting Significant Changes page...
Deleting Settle page...
Deleting Discovery page...
Deleting Isles page..

## Create new pages using Discogs info

In [46]:
def create_page(data: dict):
    create_url = 'https://api.notion.com/v1/pages'

    payload = {'parent': {'database_id': database_id}, 'properties': data}

    res = requests.post(create_url, headers=headers, json=payload)
    print(res.status_code)
    
    return res

In [47]:
def setup_data(df):
    for index, row in df.iterrows():
        # Extract data from df
        album = row['album']
        artist = row['artist']
        url = row['url']
        tag = row['tags']
        image = row['image']

        # Defining color for tags
        if tag == 'wish':
            color = 'purple'
        elif tag == 'collection':
            color = 'green'
        else:
            color = 'blue'

        # Setting up data for new page
        data = {
            'Album': {'title': [{'text': {'content': album}}]},
            'Artist': {'multi_select': [{'name': artist}]},
            'URL': {'url': url},
            'Tags': {'multi_select': [{'name': tag, 'color': color}]},
            'Album cover': {'files': [{'name': 'image', 'type': 'external', 'external': {'url': image}}]}
            }

        print(f'Creating {album} page...')
        create_page(data)

In [48]:
# Setup data and create new pages
setup_data(discogs_collection)
setup_data(discogs_wantlist)

Creating Oxnard page...
200
Creating Sometimes I Might Be Introvert page...
200
Creating Parcels page...
200
Creating Live Vol.1 page...
200
Creating Alligator page...
200
Creating Suddenly page...
200
Creating Cyclorama page...
200
Creating Terra Firma page...
200
Creating Significant Changes page...
200
Creating Isles page...
200
Creating Capricorn Sun page...
200
Creating Mr. Morale & The Big Steppers page...
200
Creating Sand page...
200
Creating Settle page...
200
Creating Wide Awake! page...
200
Creating Are We Still Dreaming? page...
200
Creating Venice page...
200
Creating Remember page...
200
Creating Discovery page...
200
Creating Cut The Wire  page...
200
Creating For Ever page...
200
Creating 99.9% page...
200
Creating Ventura page...
200
Creating Not Waving, But Drowning page...
200
Creating Day/Night page...
200
Creating Random Access Memories page...
200
Creating Kingdoms In Colour page...
200
Creating No Sleep in LA page...
200
Creating Mordechai page...
200
Creating Hu